Data preprocessing

In [34]:
with open('endata.txt', 'r') as text_file:
    eng_sent = text_file.readlines()
eng_sent = [sent.replace('\n','') for sent in eng_sent]

In [35]:
with open('rusdata.txt', 'r') as text_file:
  ru_sent = text_file.readlines()

In [36]:
ru_sent = [sent.replace('\n','') for sent in ru_sent]

In [37]:
import pandas as pd

df = pd.DataFrame({'en':eng_sent,'ru':ru_sent})

In [38]:
training_data = df.iloc[:2643]
training_list = []

validation_data = df.iloc[2643:]
validation_list = []

In [39]:
for i, row in training_data.iterrows():
  temp_dict = {
          'en': row.en,
          'ru': row.ru
      }
  training_list.append(temp_dict)

temp_dict_1 = {
    'translation': training_list
}

for i, row in validation_data.iterrows():
  temp_dict = {
          'en': row.en,
          'ru': row.ru
      }
  validation_list.append(temp_dict)

temp_dict_2 = {
    'translation': validation_list
}

In [40]:
training_dataset = {}
training_dataset['train'] = temp_dict_1
training_dataset['validation'] = temp_dict_2

In [41]:
from transformers import pipeline

model_checkpoint = "Helsinki-NLP/opus-mt-en-ru"
translator = pipeline("translation", model=model_checkpoint)
translator("this is a test")

[{'translation_text': 'Это тест.'}]

In [10]:
pip install sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 12.1 MB/s eta 0:00:00


In [42]:
from transformers import AutoTokenizer

model_checkpoint = "Helsinki-NLP/opus-mt-en-ru"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")

In [43]:
max_length = 128


def preprocess_function(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["ru"] for ex in examples["translation"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs

In [44]:
tokenized_validation_list = []
tokenized_train_list = []
tokenized_datasets = {}

validation_function_output = preprocess_function(training_dataset['validation'])
training_function_output = preprocess_function(training_dataset['train'])

In [45]:
for i in range(len(validation_function_output['input_ids'])):
  temp_dict = {
      'input_ids' : validation_function_output['input_ids'][i],
      'attention_mask': validation_function_output['attention_mask'][i],
      'labels': validation_function_output['labels'][i]
  }
  tokenized_validation_list.append(temp_dict)

for i in range(len(training_function_output['input_ids'])):
  temp_dict = {
      'input_ids' : training_function_output['input_ids'][i],
      'attention_mask': training_function_output['attention_mask'][i],
      'labels': training_function_output['labels'][i]
  }
  tokenized_train_list.append(temp_dict)

In [46]:
tokenized_datasets['validation'] = tokenized_validation_list
tokenized_datasets['train'] = tokenized_train_list

In [47]:
tokenized_datasets['train'][0]

{'input_ids': [32, 4686, 55477, 480, 33121, 59, 4, 4382, 3, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [1227, 3464, 229, 2190, 13320, 15384, 575, 95, 12059, 3, 0]}

Fine tuning of the model

In [48]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [49]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [50]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [51]:
batch["labels"]

tensor([[ 1227,  3464,   229,  2190,  3759,    53,  7392,    92,  4495,   809,
         38765,   776,    95,   172,  7043,     3,     0],
        [23531,    11,    44,  3464,   229,  2190, 10783, 24432,   193,   732,
           457,     3,     0,  -100,  -100,  -100,  -100]])

In [52]:
batch["decoder_input_ids"]

tensor([[62517,  1227,  3464,   229,  2190,  3759,    53,  7392,    92,  4495,
           809, 38765,   776,    95,   172,  7043,     3],
        [62517, 23531,    11,    44,  3464,   229,  2190, 10783, 24432,   193,
           732,   457,     3,     0, 62517, 62517, 62517]])

In [53]:
for i in range(1, 3):
    print(tokenized_datasets["train"][i]["labels"])

[1227, 3464, 229, 2190, 3759, 53, 7392, 92, 4495, 809, 38765, 776, 95, 172, 7043, 3, 0]
[23531, 11, 44, 3464, 229, 2190, 10783, 24432, 193, 732, 457, 3, 0, -100, -100, -100, -100]


In [54]:
!pip install sacrebleu

In [12]:
pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.4 MB/s eta 0:00:00


In [55]:
import evaluate

metric = evaluate.load("sacrebleu")

In [56]:
import numpy as np


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [57]:
from huggingface_hub import notebook_login

notebook_login()

In [13]:
pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [14]:
pip install accelerate -U

In [58]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"Gopal-finetuned-custom-en-to-ru",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=200,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

In [59]:
 from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [60]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [31]:
trainer.evaluate(max_length=max_length)

{'eval_loss': 1.2806780338287354,
 'eval_bleu': 31.288952328211977,
 'eval_runtime': 2667.5783,
 'eval_samples_per_second': 0.387,
 'eval_steps_per_second': 0.006}

In [61]:
trainer.train()

Step,Training Loss
500,0.868100
1000,0.453300
1500,0.249700
2000,0.139900
2500,0.084900
3000,0.058100
3500,0.043400
4000,0.034300
4500,0.028200
5000,0.023900


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62517]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62517]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strate

Step,Training Loss
500,0.868100
1000,0.453300
1500,0.249700
2000,0.139900
2500,0.084900
3000,0.058100
3500,0.043400
4000,0.034300
4500,0.028200
5000,0.023900


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62517]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62517]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strate

TrainOutput(global_step=8400, training_loss=0.12469503470829554, metrics={'train_runtime': 3742.4084, 'train_samples_per_second': 141.246, 'train_steps_per_second': 2.245, 'total_flos': 1.02743295787008e+16, 'train_loss': 0.12469503470829554, 'epoch': 200.0})

In [62]:
trainer.push_to_hub(tags="translation", commit_message="Training complete")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62517]], 'forced_eos_token_id': 0}


model.safetensors:   0%|          | 0.00/305M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1711693763.9a0f808d947e.1707.2:   0%|          | 0.00/9.22k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Bradpitt1234/Gopal-finetuned-custom-en-to-ru/commit/4585f6939a6ed039651dd20ad3a120a9708b8eac', commit_message='Training complete', commit_description='', oid='4585f6939a6ed039651dd20ad3a120a9708b8eac', pr_url=None, pr_revision=None, pr_num=None)

In [63]:
trainer.evaluate(max_length=max_length)

{'eval_loss': 1.8298757076263428,
 'eval_bleu': 30.233214532312058,
 'eval_runtime': 72.935,
 'eval_samples_per_second': 15.521,
 'eval_steps_per_second': 0.247,
 'epoch': 200.0}

In [64]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("translation", model="Bradpitt1234/Gopal-finetuned-custom-en-to-ru")

pipe('The digital signal processor (DSP) can be used for both noise cancellation and equalization based on the requirement.')

model.safetensors:   0%|          | 0.00/305M [00:00<?, ?B/s]

[{'translation_text': 'Цифровой сигнальный процессор (DSP) может быть использовался для шумоподавления и эквализации в зависимости от требования.'}]

In [67]:
pipe("I am going out with friends today, do not disturb me. I think weather is nice today")

[{'translation_text': 'Я встречаюсь с друзьями сегодня, не беспокойте меня. Я думаю, погода сегодня хороша.'}]